# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846844604687                   -0.70    4.8
  2   -7.852324506740       -2.26       -1.53    1.0
  3   -7.852646075143       -3.49       -2.52    3.2
  4   -7.852646677505       -6.22       -3.37    2.0
  5   -7.852646685882       -8.08       -4.76    1.2
  6   -7.852646686724       -9.07       -5.17    3.8
  7   -7.852646686729      -11.26       -5.93    1.5
  8   -7.852646686730      -12.37       -7.22    2.5
  9   -7.852646686730   +  -15.05       -7.43    2.2
 10   -7.852646686730   +  -15.05       -8.78    2.2
 11   -7.852646686730      -14.57       -9.53    3.0
 12   -7.852646686730   +  -15.05       -9.59    1.0
 13   -7.852646686730      -14.57      -11.02    2.0
 14   -7.852646686730   +  -14.45      -11.03    1.2
 15   -7.852646686730   +  -15.05      -12.19    2.5


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846842837558                   -0.70           4.8
  2   -7.852526275600       -2.25       -1.64   0.80    2.0
  3   -7.852636414924       -3.96       -2.73   0.80    1.0
  4   -7.852646507646       -5.00       -3.26   0.80    2.2
  5   -7.852646672703       -6.78       -4.12   0.80    1.0
  6   -7.852646686460       -7.86       -4.75   0.80    2.0
  7   -7.852646686723       -9.58       -5.50   0.80    1.8
  8   -7.852646686729      -11.16       -6.81   0.80    1.5
  9   -7.852646686730      -12.44       -7.17   0.80    3.0
 10   -7.852646686730      -15.05       -8.11   0.80    1.0
 11   -7.852646686730   +  -14.75       -8.89   0.80    1.8
 12   -7.852646686730   +  -14.45       -9.24   0.80    2.0
 13   -7.852646686730      -14.75      -10.31   0.80    1.2
 14   -7.852646686730      -15.05      -10.88   0.80    2.0
 15   -7.852646686730      -15.05      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.375149e+01     3.739452e+00
 * time: 0.4675459861755371
     1     1.240606e+00     2.014757e+00
 * time: 0.7329211235046387
     2    -1.634993e+00     2.110146e+00
 * time: 0.7601809501647949
     3    -3.880072e+00     2.017669e+00
 * time: 0.7999579906463623
     4    -5.268418e+00     1.784266e+00
 * time: 0.8396220207214355
     5    -6.862216e+00     1.277918e+00
 * time: 0.8816330432891846
     6    -6.966118e+00     1.626533e+00
 * time: 0.9092941284179688
     7    -7.357613e+00     1.088615e+00
 * time: 0.9373679161071777
     8    -7.382638e+00     1.880260e+00
 * time: 0.9653141498565674
     9    -7.578859e+00     1.624281e+00
 * time: 0.9943289756774902
    10    -7.631019e+00     1.092616e+00
 * time: 1.036102056503296
    11    -7.672122e+00     7.832488e-01
 * time: 1.0781540870666504
    12    -7.700346e+00     1.258960e+00
 * time: 1.1064269542694092
    13    -7.735334e+00     8.482296e-01
 * time: 1.13426208496

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846824814470                   -0.70    4.5
  2   -7.852309094668       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686675                   -2.54
  2   -7.852646686730      -10.26       -5.95
  3   -7.852646686730   +    -Inf      -12.31


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 5.626694552303882e-13
|ρ_newton - ρ_scfv| = 9.541130500476552e-13
|ρ_newton - ρ_dm|   = 2.0548017955722177e-10
